In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e6\train.csv")
test = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e6\test.csv")

In [3]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [4]:
test_id = test['id']

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [7]:
y = encoder.fit_transform(train['Fertilizer Name'])

In [8]:
train_cat = pd.get_dummies(train[['Soil Type','Crop Type']])
test_cat = pd.get_dummies(test[['Soil Type','Crop Type']])

In [9]:
test_cat = test_cat.reindex(columns=train_cat.columns,fill_value=0)

In [11]:
train_num = train[['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']]
test_num = test[['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']]

In [12]:
x = pd.concat([train_num,train_cat],axis=1)
x_test_final = pd.concat([test_num,test_cat],axis=1)

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
model = XGBClassifier(n_estimators=150,max_depth=10,learning_rate=0.1,random_state=42,eval_metric='mlogloss')
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, ...)

In [21]:
probs = model.predict_proba(x_test_final)

In [22]:
probs

array([[0.09570529, 0.11052267, 0.1509261 , ..., 0.18896607, 0.15663399,
        0.15013793],
       [0.12991343, 0.08946034, 0.19709559, ..., 0.13053714, 0.10012725,
        0.1897673 ],
       [0.16170067, 0.10171124, 0.127572  , ..., 0.17403881, 0.12106384,
        0.12942375],
       ...,
       [0.15104589, 0.22090086, 0.10143946, ..., 0.09914088, 0.17880142,
        0.11668371],
       [0.11791842, 0.14160909, 0.12002596, ..., 0.21515633, 0.14723743,
        0.16867225],
       [0.14640984, 0.22014113, 0.15525661, ..., 0.11342955, 0.08299384,
        0.10450768]], shape=(250000, 7), dtype=float32)

In [23]:
top_3 = np.argsort(probs,axis=1)[:,-3:][:,::-1]
top_3_labels = encoder.inverse_transform(top_3.ravel()).reshape(top_3.shape)
top_3_stg = [' '.join(row) for row in top_3_labels]

In [24]:
Optimal_Fertilizers_by_XGBoost = pd.DataFrame({'id' : test['id'],'Fertilizer Name' : top_3_stg})
Optimal_Fertilizers_by_XGBoost.to_csv("Optimal_Fertilizers_by_XGBoost.csv",index=False)
print("Saved")

Saved


In [27]:
def mapk(actual, predicted, k=3):
    score = 0.0
    for act, pred in zip(actual, predicted):
        try:
            rank = pred.index(act)
            if rank < k:
                score += 1.0 / (rank + 1)
        except ValueError:
            pass
    return score / len(actual)

# Convert y_val to original labels
y_val_labels = encoder.inverse_transform(y_test)
val_pred_strings = [[encoder.inverse_transform([i])[0] for i in row] for row in top_3]

val_score = mapk(y_val_labels, val_pred_strings, k=3)
print(f"Validation MAP@3 score: {val_score:.4f}")

Validation MAP@3 score: 0.2689
